<a href="https://colab.research.google.com/github/fasterinnerlooper/Notebooks/blob/main/training/train-codeberta-on-lcc_csharp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install torch
import torch
if not torch.cuda.is_available():
  %pip install tensorflow==2.14 intel-extension-for-tensorflow[xpu]==2.14 intel-optimization-for-horovod==0.28.1.1 torch==2.0.1 torchvision==0.15.2 intel_extension_for_pytorch==2.0.100 oneccl-bind-pt==2.0.0 --extra-index-url https://pytorch-extension.intel.com/release-whl/stable/cpu/us/ -q


In [5]:
%pip install wandb numpy datasets transformers[torch] accelerate -U -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 91.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, whi

In [7]:
use_wandb = True

if use_wandb:
    import wandb

    wandb.init(project="codeBERTa-on-csharp")

wandb: Currently logged in as: shafiq-jetha. Use `wandb login --relogin` to force relogin


In [1]:
from datasets import load_dataset
from transformers import (
    TrainingArguments,
    Trainer,
    AutoTokenizer,
    AutoModelForMaskedLM,
    DataCollatorForLanguageModeling,
)


model_name = "fasterinnerlooper/codeBERTa-csharp"

# download prepare the data
# dataset = load_dataset("code_search_net","javascript")
dataset = load_dataset("fasterinnerlooper/LCC_csharp")
tokenizer = AutoTokenizer.from_pretrained(model_name, return_special_tokens_mask=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

...

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/838k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/482k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.21M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

In [2]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True,
)

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(30000))
small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(10000))

In [5]:
small_train_dataset.features

{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [8]:
import os

model = AutoModelForMaskedLM.from_pretrained(model_name)
model.to("cuda")

if wandb:
    os.environ["WANDB_PROJECT"] = "codeBERTa-on-csharp"
    os.environ["WANDB_LOG_MODEL"] = "true"
    os.environ["WANDB_WATCH"] = "false"


training_args = TrainingArguments(
    output_dir="models",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    warmup_steps=500,
    # push_to_hub=True,
)

# pass "wandb" to the 'report_to' parameter to turn on wandb logging
if wandb:
    training_args.report_to = ["wandb"]

# define the trainer and start training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.899700,1.701473


In [ ]:
model.push_to_hub("codeBERTa-csharp")
tokenizer.push_to_hub("codeBERTa-csharp")

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

if wandb:
    wandb.finish()

In [ ]:
from transformers import pipeline

text = """
//
// Author:
//   Jb Evain (jbevain@gmail.com)
//
// Copyright (c) 2008 - 2015 Jb Evain
// Copyright (c) 2008 - 2011 Novell, Inc.
//
// Licensed under the MIT/X11 license.
//

namespace Mono.Cecil.Cil {

	public sealed class VariableDefinition : VariableReference {

		public bool IsPinned {
			get { return variable_type.IsPinned; }
		}

		public VariableDefinition (TypeReference variableType)
			: base (variableType)
		{
		}

		public override VariableDefinition Resolve ()
		{
			<mask> <mask>;
		}
	}
}"""

# should produce 'return this'
pipe = pipeline("fill-mask", model=model, tokenizer=tokenizer, device="cuda")
res = pipe(text)
for r in res[0]:
	if(r['score'] > .50):
		for x in (pipe(r['sequence'])):
			print(str(x['score']) + ": " + str(x['token_str']))
			print(r['token_str'] + x['token_str'])